In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.appName('Pass_columns').getOrCreate()

25/02/08 15:40:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
Shot = spark.read.csv('Dataset/events/Spain - La Liga.csv', header=True, inferSchema=True, sep=';')

In [6]:
Shot_assist = Shot.filter(Shot.pass_assisted_shot_id.isNotNull())

In [ ]:
# get only the columns that are not null when the pass is an assist
pass_cols = []
for cols in Shot_assist.columns:
    if Shot_assist.filter(F.col(cols).isNull()).count() != Shot_assist.count():
        pass_cols.append(cols)

In [6]:
pass_cols = ['counterpress', 'duration', 'id', 'index', 'location', 'match_id', 'minute', 'pass_aerial_won', 'pass_angle',
             'pass_assisted_shot_id', 'pass_body_part', 'pass_cross', 'pass_cut_back', 'pass_deflected', 'pass_end_location',
             'pass_goal_assist', 'pass_height', 'pass_length', 'pass_recipient', 'pass_recipient_id', 'pass_shot_assist',
             'pass_switch', 'pass_type', 'period', 'play_pattern', 'player', 'player_id', 'position', 'possession', 'possession_team',
             'possession_team_id', 'related_events', 'second', 'team', 'team_id', 'timestamp', 'type', 'under_pressure', 'competition',
             'season', 'pass_technique', 'pass_through_ball', 'off_camera', 'pass_inswinging', 'pass_outswinging', 'pass_straight',
             'pass_no_touch']

### Splitting Columns

In [ ]:
identify_cols = ['id', 'player_id', 'type', 'pass_assisted_shot_id']

features_cols = ['counterpress', 'pass_aerial_won', 'pass_angle', 'pass_body_part', 'pass_cross', 'pass_cut_back',
                 'pass_deflected', 'pass_height', 'pass_length', 'pass_shot_assist', 'pass_switch', 'pass_type',
                 'play_pattern', 'under_pressure', 'pass_technique', 'pass_through_ball', 'pass_inswinging', 'pass_outswinging',
                 'pass_straight', 'pass_no_touch']

dropped_cols  = ['duration', 'match_id', 'period', 'position', 'team_id', 'index', 'minute', 'pass_end_location', 'pass_goal_assist',
                 'pass_recipient', 'pass_recipient_id', 'player_id', 'player', 'possession', 'possession_team', 'possession_team_id',
                 'related_events', 'second', 'team', 'timestamp', 'location', 'off_camera', 'competition', 'season']

In [ ]:
Shot_assist.select(identify_cols + features_cols).show(10)

+--------------------+---------+----+------------+---------------+-----------+---------------------+--------------+----------+-------------+--------------+-----------+-----------+----------------+-----------+---------+--------------+--------------+--------------+-----------------+---------------+----------------+-------------+-------------+
|                  id|player_id|type|counterpress|pass_aerial_won| pass_angle|pass_assisted_shot_id|pass_body_part|pass_cross|pass_cut_back|pass_deflected|pass_height|pass_length|pass_shot_assist|pass_switch|pass_type|  play_pattern|under_pressure|pass_technique|pass_through_ball|pass_inswinging|pass_outswinging|pass_straight|pass_no_touch|
+--------------------+---------+----+------------+---------------+-----------+---------------------+--------------+----------+-------------+--------------+-----------+-----------+----------------+-----------+---------+--------------+--------------+--------------+-----------------+---------------+----------------+

In [11]:
Shot_assist.select(identify_cols + features_cols).printSchema()

root
 |-- id: string (nullable = true)
 |-- player_id: double (nullable = true)
 |-- type: string (nullable = true)
 |-- counterpress: boolean (nullable = true)
 |-- pass_aerial_won: boolean (nullable = true)
 |-- pass_angle: double (nullable = true)
 |-- pass_assisted_shot_id: string (nullable = true)
 |-- pass_body_part: string (nullable = true)
 |-- pass_cross: boolean (nullable = true)
 |-- pass_cut_back: boolean (nullable = true)
 |-- pass_deflected: boolean (nullable = true)
 |-- pass_height: string (nullable = true)
 |-- pass_length: double (nullable = true)
 |-- pass_shot_assist: boolean (nullable = true)
 |-- pass_switch: boolean (nullable = true)
 |-- pass_type: string (nullable = true)
 |-- play_pattern: string (nullable = true)
 |-- under_pressure: boolean (nullable = true)
 |-- pass_technique: string (nullable = true)
 |-- pass_through_ball: boolean (nullable = true)
 |-- pass_inswinging: boolean (nullable = true)
 |-- pass_outswinging: boolean (nullable = true)
 |-- pass_

### Selecting Features

#### Boolean variables

In [ ]:
bool_vars = ['counterpress','pass_aerial_won','pass_cross','pass_cut_back','pass_deflected','pass_shot_assist','pass_switch',
             'under_pressure','pass_through_ball','pass_inswinging','pass_outswinging','pass_straight','pass_no_touch']
for va in bool_vars:
    Shot_assist.groupBy(va).count().show()

+------------+-----+
|counterpress|count|
+------------+-----+
|        NULL|10267|
|        true|   17|
+------------+-----+



+---------------+-----+
|pass_aerial_won|count|
+---------------+-----+
|           NULL|10088|
|           true|  196|
+---------------+-----+



+----------+-----+
|pass_cross|count|
+----------+-----+
|      NULL| 8144|
|      true| 2140|
+----------+-----+



+-------------+-----+
|pass_cut_back|count|
+-------------+-----+
|         NULL| 9977|
|         true|  307|
+-------------+-----+



+--------------+-----+
|pass_deflected|count|
+--------------+-----+
|          NULL|10270|
|          true|   14|
+--------------+-----+



...

+-----------------+-----+
|pass_through_ball|count|
+-----------------+-----+
|             NULL| 9581|
|             true|  703|
+-----------------+-----+



+---------------+-----+
|pass_inswinging|count|
+---------------+-----+
|           NULL|10015|
|           true|  269|
+---------------+-----+



+----------------+-----+
|pass_outswinging|count|
+----------------+-----+
|            NULL| 9770|
|            true|  514|
+----------------+-----+



+-------------+-----+
|pass_straight|count|
+-------------+-----+
|         NULL|10223|
|         true|   61|
+-------------+-----+



+-------------+-----+
|pass_no_touch|count|
+-------------+-----+
|         NULL|10231|
|         true|   53|
+-------------+-----+



Since `counterpress` and `pass_deflected` have very few `true` values, we will drop them.

#### Categorical variables

In [19]:
categorical_variables = ['pass_body_part','pass_height','pass_type','play_pattern','pass_technique']
for j in categorical_variables:
    Shot_assist.groupBy(j).count().show()

+--------------+-----+
|pass_body_part|count|
+--------------+-----+
|     Drop Kick|   10|
|    Right Foot| 6162|
|          NULL|  268|
|     Left Foot| 3527|
|          Head|  196|
|      No Touch|   53|
|         Other|   62|
|    Keeper Arm|    6|
+--------------+-----+



+-----------+-----+
|pass_height|count|
+-----------+-----+
|Ground Pass| 5437|
|   Low Pass| 1248|
|  High Pass| 3599|
+-----------+-----+



+------------+-----+
|   pass_type|count|
+------------+-----+
|    Throw-in|   72|
|        NULL| 8391|
|   Goal Kick|    6|
|      Corner|  883|
|Interception|   28|
|   Free Kick|  434|
|    Recovery|  470|
+------------+-----+



+--------------+-----+
|  play_pattern|count|
+--------------+-----+
|  Regular Play| 3945|
|   From Keeper|  163|
|From Goal Kick|  419|
| From Throw In| 1816|
|         Other|    6|
|   From Corner| 1588|
| From Kick Off|  127|
|  From Counter|  558|
|From Free Kick| 1662|
+--------------+-----+



+--------------+-----+
|pass_technique|count|
+--------------+-----+
|    Inswinging|  269|
|          NULL| 8737|
|   Outswinging|  514|
|      Straight|   61|
|  Through Ball|  703|
+--------------+-----+



`pass_body_part` will be kept for now.</br>
`pass_height` will be encoded (0: Ground Pass, 1: Low Pass, 2: High Pass)</br>
`pass_type` will be kept and compared with `play_pattern`</br>
`play_pattern` beside being compared with `pass_type` it will also be compared with `play_pattern` of the shot</br>
`pass_technique` apparantly it holds same information as the boolean variables, therefore this theory will be tested
if found to be true, it will be dropped.

#### Selected features

In [61]:
PASS_FEATURES = ['id', 'pass_assisted_shot_id', 'player_id', 'under_pressure', 'pass_body_part', 'pass_height',
                 'pass_type', 'play_pattern','pass_technique', 'pass_length', 'pass_angle', 
                 'pass_aerial_won', 'pass_cross', 'pass_cut_back', 'pass_switch', 'pass_through_ball', 'pass_inswinging',
                 'pass_outswinging', 'pass_straight', 'pass_no_touch']
Passes = Shot_assist.select(PASS_FEATURES)
Passes.show()

+--------------------+---------------------+---------+--------------+--------------+-----------+---------+--------------+--------------+-----------+-----------+---------------+----------+-------------+-----------+-----------------+---------------+----------------+-------------+-------------+
|                  id|pass_assisted_shot_id|player_id|under_pressure|pass_body_part|pass_height|pass_type|  play_pattern|pass_technique|pass_length| pass_angle|pass_aerial_won|pass_cross|pass_cut_back|pass_switch|pass_through_ball|pass_inswinging|pass_outswinging|pass_straight|pass_no_touch|
+--------------------+---------------------+---------+--------------+--------------+-----------+---------+--------------+--------------+-----------+-----------+---------------+----------+-------------+-----------+-----------------+---------------+----------------+-------------+-------------+
|92f50ac4-2f1c-422...| c5341577-e1ca-474...|   6826.0|          NULL|     Left Foot|Ground Pass|     NULL| From Throw In|

### Preprocessing

In [62]:
# Pass height
Passes = Passes.withColumn('pass_height',F.when(F.col('pass_height')=='Ground Pass', 0)\
    .when(F.col('pass_height')=='Low Pass', 1).otherwise(2))

In [63]:
# Pass type
Passes.groupBy('pass_type','play_pattern').count().orderBy('pass_type').show(26)

+------------+--------------+-----+
|   pass_type|  play_pattern|count|
+------------+--------------+-----+
|        NULL|  Regular Play| 3771|
|        NULL|   From Keeper|  160|
|        NULL|From Goal Kick|  387|
|        NULL| From Throw In| 1657|
|        NULL|         Other|    6|
|        NULL|   From Corner|  639|
|        NULL| From Kick Off|  120|
|        NULL|  From Counter|  490|
|        NULL|From Free Kick| 1161|
|      Corner|   From Corner|  883|
|   Free Kick|From Free Kick|  434|
|   Goal Kick|From Goal Kick|    6|
|Interception|  Regular Play|   14|
|Interception|  From Counter|    8|
|Interception| From Throw In|    2|
|Interception|From Free Kick|    3|
|Interception|   From Corner|    1|
|    Recovery|From Free Kick|   64|
|    Recovery|   From Corner|   65|
|    Recovery|  Regular Play|  160|
|    Recovery| From Throw In|   85|
|    Recovery|From Goal Kick|   26|
|    Recovery|  From Counter|   60|
|    Recovery| From Kick Off|    7|
|    Recovery|   From Keeper

`pass_type` overlaps with `play_pattern`, only `play_pattern` will be kept.

In [64]:
# Pass technique
Passes.groupBy('pass_technique','pass_straight','pass_outswinging','pass_inswinging','pass_through_ball').count().show()

+--------------+-------------+----------------+---------------+-----------------+-----+
|pass_technique|pass_straight|pass_outswinging|pass_inswinging|pass_through_ball|count|
+--------------+-------------+----------------+---------------+-----------------+-----+
|      Straight|         true|            NULL|           NULL|             NULL|   61|
|   Outswinging|         NULL|            true|           NULL|             NULL|  514|
|          NULL|         NULL|            NULL|           NULL|             NULL| 8737|
|    Inswinging|         NULL|            NULL|           true|             NULL|  269|
|  Through Ball|         NULL|            NULL|           NULL|             true|  703|
+--------------+-------------+----------------+---------------+-----------------+-----+



The information stored in `pass_technique` is the same stored in the boolean variables, therefore it will be dropped. 

In [65]:
# rename columns to avoid confusion when joining with shots dataframe
Passes = Passes.withColumnsRenamed({'id':'pass_id',
                                    'player_id':'pass_player_id',
                                    'play_pattern':'pass_play_pattern',
                                    'under_pressure':'pass_under_pressure'})

In [66]:
Passes.drop('pass_type','pass_technique').show(10)

+--------------------+---------------------+--------------+-------------------+--------------+-----------+-----------------+-----------+-----------+---------------+----------+-------------+-----------+-----------------+---------------+----------------+-------------+-------------+
|             pass_id|pass_assisted_shot_id|pass_player_id|pass_under_pressure|pass_body_part|pass_height|pass_play_pattern|pass_length| pass_angle|pass_aerial_won|pass_cross|pass_cut_back|pass_switch|pass_through_ball|pass_inswinging|pass_outswinging|pass_straight|pass_no_touch|
+--------------------+---------------------+--------------+-------------------+--------------+-----------+-----------------+-----------+-----------+---------------+----------+-------------+-----------+-----------------+---------------+----------------+-------------+-------------+
|92f50ac4-2f1c-422...| c5341577-e1ca-474...|        6826.0|               NULL|     Left Foot|          0|    From Throw In|   40.08441| 0.06490869|         

In [7]:
pass_c = ['pass_assisted_shot_id', 'pass_height',
          'pass_length', 'pass_angle', 'pass_aerial_won', 'under_pressure', 'pass_cross', 'pass_cut_back',
          'pass_switch', 'pass_through_ball', 'pass_inswinging', 'pass_outswinging', 'pass_straight', 'pass_no_touch']

def Pass_cols(df, pass_c):
    df = df.select(pass_c)
    df = df.withColumn('pass_height',F.when(F.col('pass_height')=='Ground Pass', 0)\
        .when(F.col('pass_height')=='Low Pass', 1).otherwise(2))
    df = df.withColumnsRenamed({'id':'pass_id',
                                    'player_id':'pass_player_id',
                                    'play_pattern':'pass_play_pattern',
                                    'under_pressure':'pass_under_pressure'})
    return df

In [8]:
df = Pass_cols(Shot_assist,pass_c)

In [9]:
df.show()

+---------------------+-----------+-----------+-----------+---------------+-------------------+----------+-------------+-----------+-----------------+---------------+----------------+-------------+-------------+
|pass_assisted_shot_id|pass_height|pass_length| pass_angle|pass_aerial_won|pass_under_pressure|pass_cross|pass_cut_back|pass_switch|pass_through_ball|pass_inswinging|pass_outswinging|pass_straight|pass_no_touch|
+---------------------+-----------+-----------+-----------+---------------+-------------------+----------+-------------+-----------+-----------------+---------------+----------------+-------------+-------------+
| c5341577-e1ca-474...|          0|   40.08441| 0.06490869|           NULL|               NULL|      NULL|         NULL|       NULL|             true|           NULL|            NULL|         NULL|         NULL|
| 1aedaf9e-bc12-4d0...|          0|    42.3495|  1.3372719|           NULL|               NULL|      true|         NULL|       true|             NULL|  

In [10]:
from modules.alt_preprocessing import Preprocessing

In [11]:
Shots = Preprocessing(spark, Shot)

In [12]:
Shots.show(10)

25/02/08 15:41:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+---------+---------------+---------------+----------------+----------+-------------------+-------+-------+-------+-----------+------------+-------+-----------+-------+------+-----------+-------+-------+---------------------+----------------+-------------+------------------+------------------+------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-------------------+-----------------+----+
|                  id|player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|from_rp|from_fk|from_ti|from_corner|from_counter|from_gk|from_keeper|from_ko|header|corner_type|fk_type|pk_type|half_volley_technique|volley_technique|lob_technique|overhead_technique|backheel_technique|diving_h_technique|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|players_inside_area|shot_statsbomb_xg|goal|
+--------------------+---------+--------------

In [15]:
final_df = Shots.join(df, Shots.id == df.pass_assisted_shot_id, how='left')

In [16]:
final_df.show()

+--------------------+---------+---------------+---------------+----------------+----------+-------------------+-------+-------+-------+-----------+------------+-------+-----------+-------+------+-----------+-------+-------+---------------------+----------------+-------------+------------------+------------------+------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-------------------+-----------------+----+---------------------+-----------+-----------+-------------+---------------+-------------------+----------+-------------+-----------+-----------------+---------------+----------------+-------------+-------------+
|                  id|player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|from_rp|from_fk|from_ti|from_corner|from_counter|from_gk|from_keeper|from_ko|header|corner_type|fk_type|pk_type|half_volley_technique|volley_technique|lob_technique|overhead_technique|backheel_techni

In [ ]:
df.orderBy('pass_angle',ascending=False).show()

+--------------------+---------------------+--------------+-------------------+--------------+-----------+-----------------+-----------+----------+---------------+----------+-------------+-----------+-----------------+---------------+----------------+-------------+-------------+
|             pass_id|pass_assisted_shot_id|pass_player_id|pass_under_pressure|pass_body_part|pass_height|pass_play_pattern|pass_length|pass_angle|pass_aerial_won|pass_cross|pass_cut_back|pass_switch|pass_through_ball|pass_inswinging|pass_outswinging|pass_straight|pass_no_touch|
+--------------------+---------------------+--------------+-------------------+--------------+-----------+-----------------+-----------+----------+---------------+----------+-------------+-----------+-----------------+---------------+----------------+-------------+-------------+
|628a547d-873b-48c...| 35db9fba-2fe6-463...|        5246.0|               NULL|    Right Foot|          0|    From Throw In|        5.5| 3.1415927|           NU

In [1]:
# Pass angle is between -pi and pi
# if shot has no assist, Pass angle will be 4.
# pass_id will be use to identify if pass exists, if notNull then 1, else 0
# pass_height will be -1 if no pass 
# play_pattern is same in pass and shot, drop the pass column